In [3]:
%pylab inline
import pandas as pd
import numpy as np
import requests
import glob
import json

Populating the interactive namespace from numpy and matplotlib


In [4]:
import requests
import json

cases_endpt = 'https://api.gdc.cancer.gov/cases'

# The 'fields' parameter is passed as a comma-separated string of single names.
fields = [
    "submitter_id",
    "case_id",
    "diagnoses.*",
    "samples",
    "files"
    ]

fields = ','.join(fields)


filters = {
    "op": "in",
    "content":{
        "field": "cases.project.project_id",
        "value": "TCGA-LIHC"
        }
    }

# With a GET request, the filters parameter needs to be converted
# from a dictionary to JSON-formatted string

params = {
    "filters": json.dumps(filters),
    "fields": fields,
    "format": "JSON",
    "size": "500",
    "return_manifest": True,
    "expand": "diagnoses,samples.*,files"
    }

response = requests.get(cases_endpt, params = params).json()

if response['data']['pagination']['pages'] != 1:
    print("you need to figure out how to deal w/ pagination")


In [5]:
case = response['data']['hits'][0]

In [7]:
case['samples']

[{'composition': None,
  'created_datetime': None,
  'current_weight': None,
  'days_to_collection': 78,
  'days_to_sample_procurement': None,
  'freezing_method': None,
  'initial_weight': None,
  'intermediate_dimension': None,
  'is_ffpe': False,
  'longest_dimension': None,
  'oct_embedded': 'false',
  'pathology_report_uuid': None,
  'portions': [{'analytes': [{'a260_a280_ratio': None,
      'aliquots': [{'aliquot_id': 'b302e550-799e-447e-8744-f32130f8063f',
        'amount': None,
        'analyte_type': None,
        'analyte_type_id': None,
        'center': {'center_id': '5069ce55-a23f-57c4-a28c-70a3c3cb0e4c',
         'center_type': 'CGCC',
         'code': '01',
         'name': 'Broad Institute of MIT and Harvard',
         'namespace': 'broad.mit.edu',
         'short_name': 'BI'},
        'concentration': 0.15,
        'created_datetime': None,
        'source_center': '23',
        'state': 'live',
        'submitter_id': 'TCGA-ED-A7PZ-10A-01D-A33P-01',
        'updated_

In [8]:
slide_uuids = []
slides = []
finfo = []

for case in response['data']['hits']:
    for sample in case.get('samples',[]):
        for portion in sample.get('portions',[]):
            for slide in portion.get('slides',[]):
                slide_uuids.append(slide['slide_id'])
                slides.append(slide)
                import pdb; pdb.set_trace()
    for f in case.get('files',[]):
        finfo.append(f)
        

> <ipython-input-8-6b3b8724ec54>(8)<module>()
-> for slide in portion.get('slides',[]):
(Pdb) portion.keys()
dict_keys(['creation_datetime', 'portion_number', 'weight', 'updated_datetime', 'created_datetime', 'analytes', 'submitter_id', 'slides', 'state', 'portion_id', 'is_ffpe'])
(Pdb) portion['slides']
[{'percent_neutrophil_infiltration': 0.0, 'percent_monocyte_infiltration': 0.0, 'percent_normal_cells': 0.0, 'updated_datetime': '2018-02-23T16:24:50.088756-06:00', 'created_datetime': None, 'percent_eosinophil_infiltration': None, 'percent_tumor_nuclei': 80.0, 'percent_lymphocyte_infiltration': 2.0, 'submitter_id': 'TCGA-ED-A7PZ-01A-01-TSA', 'state': 'live', 'section_location': 'TOP', 'percent_granulocyte_infiltration': None, 'percent_necrosis': 0.0, 'slide_id': '7d5ac320-f0a8-4cdb-812a-0b6969f120d0', 'number_proliferating_cells': None, 'percent_stromal_cells': 0.0, 'percent_inflam_infiltration': None, 'percent_tumor_cells': 100.0}]
(Pdb) portion['slides'][0]
{'percent_neutrophil_infi

(Pdb) case['samples'][0].keys()
dict_keys(['sample_type_id', 'updated_datetime', 'time_between_excision_and_freezing', 'oct_embedded', 'tumor_code_id', 'submitter_id', 'intermediate_dimension', 'sample_id', 'is_ffpe', 'pathology_report_uuid', 'created_datetime', 'tumor_descriptor', 'sample_type', 'state', 'current_weight', 'composition', 'time_between_clamping_and_freezing', 'shortest_dimension', 'tumor_code', 'tissue_type', 'days_to_sample_procurement', 'freezing_method', 'portions', 'preservation_method', 'days_to_collection', 'initial_weight', 'longest_dimension'])
(Pdb) case['samples'][0]['tissue_type']
(Pdb) case['samples'][0]['sample_id']
'bebe80e9-f3ef-4269-bc98-0d7712923d60'
(Pdb) case['samples'][0]['sample_type']
'Blood Derived Normal'
(Pdb) case['samples'][0]['sample_type_id']
'10'
(Pdb) slide
{'percent_neutrophil_infiltration': 0.0, 'percent_monocyte_infiltration': 0.0, 'percent_normal_cells': 0.0, 'updated_datetime': '2018-02-23T16:24:50.088756-06:00', 'created_datetime': N

BdbQuit: 

In [ ]:
with open('liver_data.json', 'w') as json_file:
    json.dump(response['data']['hits'], json_file)

In [ ]:
import requests
import json

cases_endpt = 'https://api.gdc.cancer.gov/legacy/files'

# The 'fields' parameter is passed as a comma-separated string of single names.
fields = [
    "files.data_format",
    "files.file_id",
    "files.file_name",
    "files.md5sum"
    ]

fields = ','.join(fields)

filters = {
    "op": "and",
    "content": [
        {
            "op":"=",
            "content":{
                "field":"files.data_format",
                "value":[
                    "SVS"
                ]
            },
        },
        {
            "op":"=",
            "content":{
                "field":"cases.project.project_id",
                "value":[
                    "TCGA-LIHC"
                ]
            }
        }
    ]
}

# With a GET request, the filters parameter needs to be converted
# from a dictionary to JSON-formatted string

params = {
    "filters": json.dumps(filters),
    "fields": fields,
    "format": "JSON",
    "size": "500"
}

response = requests.get(cases_endpt, params = params).json()

if response['data']['pagination']['pages'] != 1:
    print("you need to figure out how to deal w/ pagination")
    print(response['data']['pagination'])


In [ ]:
len(response['data']['hits'])

In [ ]:
files_df = pd.DataFrame(response['data']['hits'])

In [ ]:
manifest_df = files_df[['id','file_name','md5sum','file_size','state']]
manifest_df.columns = ['id','filename','md5','size','state']
manifest_df.head()

In [ ]:
manifest_df.to_csv('manifest.txt', sep='\t', index=False)

In [ ]:
manifest_df.to_csv?